In [32]:
import glob
import json
import ipaddress

In [51]:
def read_hosts(path):
    nws = []
    for h in glob.glob(path):
        with open(h) as fp:
            content = json.load(fp)
            nws.append(ipaddress.ip_network(content['hostInterfaces']['eth0']['prefix'], strict=False))
    return nws

In [111]:
def find_in_lines(needle, search):
    diff_specs = []
    for i, l in enumerate(search.splitlines()):
        # print(needle.__str__())
        if needle.__str__() in l:
            diff_specs.append(l)
            print(f"Found {needle} in line {i}:", l)
    return diff_specs

In [110]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/NetworkC-zoo"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
for nw in networks:
    nw_name = nw.split("/")[-1]
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
    host_pfxs_count = len(read_hosts(nw + "/hosts/host*.json"))
    # read diff
    with open(NETWORKS_DIR + f"/origin/not-in-{nw_name}.csv", 'r') as fp:
        diff_host_count = 0
        content = fp.read().__str__()
        for pfx in orig_host_prefixes:
            diff_specs = find_in_lines(pfx, content)
            diff_host_count += len(diff_specs)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(content.splitlines()), diff_host_count, host_pfxs_count, sep="\t")
    # break

origin	231	0	0	9
anonym-network-k6-w-noise	1374	5	0	36
strawman-1-6-4	1367	7	0	36
anonym-network-k6-4-w-noise	1367	7	0	36
anonym-network-k6-6	2074	5	0	54
anonym-network-k6-6-w-noise	2102	6	0	54
strawman-2-6-2	628	6	0	18
anonym-network-k6-2-w-noise-n	628	6	0	18
strawman-1	1374	5	0	36
anonym-network-k6-2-w-noise	612	5	0	18
strawman-1-6-2	628	6	0	18
anonym-network-k6-4	1343	8	0	36
strawman-1-6-6	2102	6	0	54


In [ ]:
NETWORKS_DIR = "/home/yongting/research/configs/config2spec-confmask/scenarios/confmask/bics-ospf"
networks = glob.glob(NETWORKS_DIR + "/*")
networks.remove(f"{NETWORKS_DIR}/origin")
networks.insert(0, f"{NETWORKS_DIR}/origin")
for nw in networks:
    nw_name = nw.split("/")[-1]
    # read host prefixes
    if nw_name == "origin":
        orig_host_prefixes = read_hosts(nw + "/hosts/host*.json")
    host_pfxs_count = len(read_hosts(nw + "/hosts/host*.json"))
    # read diff
    with open(NETWORKS_DIR + f"/origin/not-in-{nw_name}.csv", 'r') as fp:
        diff_host_count = 0
        content = fp.read().__str__()
        for pfx in orig_host_prefixes:
            diff_specs = find_in_lines(pfx, content)
            diff_host_count += len(diff_specs)
    with open(NETWORKS_DIR + f"/{nw_name}/policies.csv") as fp:
        specs_count = len(fp.read().splitlines()) - 1
    print(nw_name, specs_count, len(content.splitlines()), diff_host_count, host_pfxs_count, sep="\t")
    # break